# Training Siamese

In [5]:
import torch as th
from torchvision import transforms
from torch.utils.data import DataLoader
from siamese import Siamese
import os
import pandas as pd
from dataset import CarDataset


data_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])


## Define dataset

In [6]:
dataset = pd.read_csv('patches/gt_car_patches_annotations.csv', delimiter=',')

train = dataset[(dataset.SEQ == 'S01') | (dataset.SEQ == 'S04')]
test = dataset[(dataset.SEQ == 'S03')]

# Dataset
trainSet = CarDataset(train, transform=data_transforms)
testSet = CarDataset(test, transform=data_transforms)

trainLoader = DataLoader(trainSet, batch_size=256, shuffle=False, num_workers=4)
testLoader = DataLoader(testSet, batch_size=10, shuffle=False, num_workers=4)

print(f"Train size: {train.shape[0]} Test size {test.shape[0]}")

Train size: 38074 Test size 6174


In [ ]:
class ContrastiveLoss(th.nn.Module):
    """
    Contrastive loss function.
    Based on:
    """

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, x0, x1, y):
        # euclidian distance
        diff = x0 - x1
        dist_sq = th.sum(th.pow(diff, 2), 1)
        dist = th.sqrt(dist_sq)

        mdist = self.margin - dist
        dist = th.clamp(mdist, min=0.0)
        loss = y * dist_sq + (1 - y) * th.pow(dist, 2)
        loss = th.sum(loss) / 2.0 / x0.size()[0]
        return loss

In [7]:
gpu_ids = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_ids
print("use gpu:", gpu_ids, "to train.")

cuda = True
epochs = 10

# Declare Siamese Network
net = Siamese().cuda()
# Decalre Loss Function
criterion = ContrastiveLoss()
# Declare Optimizer
optimizer = th.optim.Adam(net.parameters(), lr=1e-3, weight_decay=0.0005)

#train the model
def train():
    loss=[]
    counter=[]
    iteration_number = 0
    for epoch in range(1, epochs):

        for i, data in enumerate(trainLoader,0):

            img0, img1 , label = data
            img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
            optimizer.zero_grad()
            output1, output2 = net(img0,img1)
            loss_contrastive = criterion(output1,output2,label)
            loss_contrastive.backward()
            optimizer.step()


            if i % 100 == 99:
              th.save(net.state_dict(), f"drive/MyDrive/model-epoch-{epoch}-check-{i}-loss-{loss_contrastive.item()}.pt")

            # Every 10 batches print out the loss
            if i % 10 == 0 :
              print(f"Epoch number {epoch} It: {i} Current loss {loss_contrastive.item()}\n")
              iteration_number += 10

              counter.append(iteration_number)
              loss.append(loss_contrastive.item())

    return net

#set the device to cuda
device = th.device('cuda' if th.cuda.is_available() else 'cpu')
model = train()
th.save(model.state_dict(), "drive/MyDrive/model.pt")
print("Model Saved Successfully")

use gpu: 0 to train.


ValueError: labels must be a 1D tensor of shape (batch_size,)